In [1]:
import tensorflow as tf
import numpy as np

input Data init

In [2]:
#make hello example
#hell -> ello
char_set = ['h', 'e', 'l', 'o']

x_one_hot = [[
    [1, 0, 0, 0], #h 0
    [0, 1, 0, 0], #e 1
    [0, 0, 1, 0], #l 2
    [0, 0, 1, 0], #l 2
]]

y_data = [[1, 2, 2, 3]]

initialize LSTM Parameters

In [3]:
num_classes = 4
input_dim = 4
hidden_size = 12 #output from LSTM
batch_size = 1 #one sentence
sequence_length = 4
learning_rate = 0.1

Init Placeholder

In [4]:
X = tf.placeholder(tf.float32, [None, sequence_length, input_dim]) #X_one_hot
Y = tf.placeholder(tf.int32, [None, sequence_length]) #Y_label

Make LSTM Cell

In [5]:
cell = tf.contrib.rnn.BasicLSTMCell(num_units = hidden_size, state_is_tuple=True)
initial_state = cell.zero_state(batch_size, tf.float32)

outputs, _states = tf.nn.dynamic_rnn(cell, X, initial_state = initial_state, dtype = tf.float32)

Instructions for updating:
This class is deprecated, please use tf.nn.rnn_cell.LSTMCell, which supports all the feature this cell currently has. Please replace the existing code with tf.nn.rnn_cell.LSTMCell(name='basic_lstm_cell').


set the fully connected Layer

In [6]:
#FC Layer
X_for_fc = tf.reshape(outputs, [-1, hidden_size])

#fc_w = tf.get_variable("fc_w", [hidden_size, num_classes])
#fc_b = tf.get_variable("fc_b", [num_classes])
#outputs = tf.matmul(X_for_fc, fc_w) + fc_b

outputs = tf.contrib.layers.fully_connected(inputs = X_for_fc, num_outputs = num_classes, activation_fn = None)

#reshape out for sequence_loss
outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes])

set the cost func and initialize the traninig func

In [7]:
weights = tf.ones([batch_size, sequence_length])
sequence_loss = tf.contrib.seq2seq.sequence_loss(logits = outputs, targets = Y, weights = weights)

loss = tf.reduce_mean(sequence_loss)
train = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(loss)
prediction = tf.argmax(outputs, axis = 2)

start traning

In [8]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for i in range(50):
        I, _= sess.run([loss, train], feed_dict = {X: x_one_hot, Y: y_data})
        result = sess.run(prediction, feed_dict = {X: x_one_hot})
        print(i, 'loss: ', I, "prediction: ", result, "true Y: ", y_data)
        
        #print char using dic
        result_str = [char_set[c] for c in np.squeeze(result)]
        print("prediction str: ", ''.join(result_str))

0 loss:  1.4133285 prediction:  [[2 2 2 2]] true Y:  [[1, 2, 2, 3]]
prediction str:  llll
1 loss:  1.1759284 prediction:  [[2 2 2 2]] true Y:  [[1, 2, 2, 3]]
prediction str:  llll
2 loss:  1.0274553 prediction:  [[2 2 2 2]] true Y:  [[1, 2, 2, 3]]
prediction str:  llll
3 loss:  0.8435077 prediction:  [[2 2 2 3]] true Y:  [[1, 2, 2, 3]]
prediction str:  lllo
4 loss:  0.681658 prediction:  [[1 2 2 3]] true Y:  [[1, 2, 2, 3]]
prediction str:  ello
5 loss:  0.52394295 prediction:  [[1 2 2 3]] true Y:  [[1, 2, 2, 3]]
prediction str:  ello
6 loss:  0.3453106 prediction:  [[1 2 2 3]] true Y:  [[1, 2, 2, 3]]
prediction str:  ello
7 loss:  0.20707802 prediction:  [[1 2 2 3]] true Y:  [[1, 2, 2, 3]]
prediction str:  ello
8 loss:  0.12203818 prediction:  [[1 2 2 3]] true Y:  [[1, 2, 2, 3]]
prediction str:  ello
9 loss:  0.059434764 prediction:  [[1 2 2 3]] true Y:  [[1, 2, 2, 3]]
prediction str:  ello
10 loss:  0.037036 prediction:  [[1 2 2 3]] true Y:  [[1, 2, 2, 3]]
prediction str:  ello
11 los

In [9]:
result_str

['e', 'l', 'l', 'o']